# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install citipy

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from scipy.stats import linregress
output_data_file = "output_data/cities.csv"
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data = pd.read_csv("output_data/cities.csv")
city_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kavieng,-2.5744,150.7967,81.09,79,97,11.72,PG,1630085507
1,1,jamestown,42.0970,-79.2353,86.09,65,40,3.44,US,1630085423
2,2,egvekinot,66.3167,-179.1667,37.63,99,100,7.78,RU,1630085509
3,3,puerto ayora,-0.7393,-90.3518,76.96,85,91,10.00,EC,1630085509
4,4,adamantina,-21.6853,-51.0725,69.55,47,97,13.02,BR,1630085510


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
location = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, dissipating=False, max_intensity=300, point_radius = 5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_city = city_data.loc[(city_data["Max Temp"] < 80) & (city_data["Max Temp"] > 70) \
                                 & (city_data["Wind Speed"]< 10) \
                                 & (city_data["Cloudiness"] == 0)].dropna()
                                 

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
narrowed_city.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
131,131,quelimane,-17.8786,36.8883,75.15,83,0,6.91,MZ,1630085603
188,188,karratha,-20.7377,116.8463,72.16,37,0,9.98,AU,1630085648
221,221,yingkou,40.6648,122.2283,70.02,75,0,9.80,CN,1630085674
320,320,lata,40.1629,-8.3327,79.03,64,0,7.52,PT,1630085751
333,333,sukhumi,43.0033,41.0153,76.89,70,0,3.49,GE,1630085759


In [8]:
hotel_df = narrowed_city[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
131,quelimane,MZ,-17.8786,36.8883,
188,karratha,AU,-20.7377,116.8463,
221,yingkou,CN,40.6648,122.2283,
320,lata,PT,40.1629,-8.3327,
333,sukhumi,GE,43.0033,41.0153,


In [26]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Lat,Lng,Hotel Name
131,quelimane,MZ,-17.8786,36.8883,Hotel Flamingo
188,karratha,AU,-20.7377,116.8463,ibis Styles Karratha
221,yingkou,CN,40.6648,122.2283,Yingkou Juyin Junyue Hotel
320,lata,PT,40.1629,-8.3327,Casa Maquia
333,sukhumi,GE,43.0033,41.0153,Ritsa Hotel


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))